In [193]:
from datasets import load_dataset
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML
import nltk
from gensim.models import Word2Vec
import tensorflow as tf
from tensorflow import keras
from keras import layers
import gensim.downloader as api

In [14]:
w2v = api.load('glove-wiki-gigaword-100')

In [ ]:
import re
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

In [97]:
f,c=['a','f']
print(c)
for x in enumerate(f):
    print(x)

f
(0, 'a')


In [186]:
class seq_dataset():
    def __init__(self, path, max_seq_len, word_embedding_model):
        self.path = path
        self.init_process()
        self.max_seq_len = max_seq_len 
        self.word_embedding = word_embedding_model
        
        
        
    def init_process(self):
        self.tokenize()
    
    def tokenize(self):
        with open(self.path,'r') as f:
            raw = f.read()
            self.token_seq = nltk.word_tokenize(raw)
            for idx,word in enumerate(self.token_seq):
                self.token_seq[idx] = word.lower()
            
            self.len = len(self.token_seq)
            
    def _word_converter(self,word):
        '''class function for exception handling when converting words to vectors'''
        try:
            vec = self.word_embedding[word]
            return([vec])
        except:
            
            if '-' in word:
                try:
                    vec = self.word_embedding[word.replace('-','')]
                    return([vec])
                except:  
                    pass
                
                try:
                    w1,w2 = word.split('-')
                    vec1_l = self._word_converter(w1)
                    vec2_l = self._word_converter(w2)
                    vec1_l.extend(vec2_l)
                    return(vec1_l)
                except:
                    pass
                
            else:
                print(word)
                return("ndad")
        
            
                
                
    
    def produce_batch(self,batch_size):
        seq_test = []
        for example in range(batch_size):
            start = random.randint(0,self.len-self.max_seq_len-1)
            stop = start+random.randint(1,self.max_seq_len)
            slice_ = self.token_seq[start:stop]
            
                                    
        return seq_test
        
        

In [187]:
moby_data = seq_dataset('./text_folder/mobydick.txt',100,w2v)
    

In [189]:
print(moby_data._word_converter("double-shudffle"))

shudffle
[array([ 2.2832e-01,  3.4314e-01,  3.4916e-01, -6.2292e-02,  5.8088e-01,
        7.6693e-02, -3.3078e-02,  1.8288e-01, -4.3578e-01,  4.6312e-01,
        2.2758e-01,  6.6902e-03, -9.0716e-02,  1.0973e+00,  2.9967e-01,
        1.4418e-01, -6.0248e-01,  4.7535e-01, -3.7224e-02, -3.6675e-01,
        1.1219e+00, -1.2940e-01, -5.0331e-01,  5.8335e-01,  7.9264e-01,
       -2.6178e-01, -2.3382e-01, -4.8055e-01,  8.7966e-02, -7.5332e-01,
       -1.8990e-01,  4.4444e-01,  9.3678e-03,  4.4959e-02,  6.8601e-01,
       -9.1632e-02, -6.7163e-01,  5.4273e-01,  2.6784e-01,  6.4011e-01,
        6.0244e-01, -1.9369e-01,  8.2975e-02,  1.0601e-01,  4.9757e-01,
        1.3826e-01,  4.3512e-01, -1.3030e+00, -3.0877e-01, -6.3144e-01,
       -3.1466e-01, -2.5677e-02, -4.7305e-01,  5.7616e-01, -1.7668e-01,
       -2.8022e+00,  3.9615e-01,  3.3021e-01,  1.5153e+00,  1.0390e+00,
       -3.1545e-01, -1.7695e-01, -1.2297e-01, -1.3058e-01,  3.0818e-01,
       -3.4138e-01,  3.3992e-01,  1.4005e-01, -2.1849e

In [114]:
moby_data.produce_batch(5)

['not', 'be', 'backward', ',', 'but', 'let', 'him', 'cheerfully', 'allow', 'himself', 'to', 'spend', 'and', 'be', 'spent', 'in', 'that', 'way', '.', 'and', 'the', 'man', 'that', 'has', 'anything', 'bountifully', 'laughable', 'about', 'him', ',', 'be', 'sure', 'there', 'is', 'more', 'in', 'that', 'man', 'than']
['as', 'howling', 'condition', 'as', 'the', 'coast', 'of', 'labrador', '.', 'as', 'it', 'is', ',', 'parts', 'of', 'her', 'back', 'country', 'are', 'enough', 'to', 'frighten', 'one', ',', 'they', 'look', 'so', 'bony', '.', 'the', 'town', 'itself', 'is', 'perhaps', 'the', 'dearest', 'place', 'to', 'live', 'in', ',', 'hi', 'all', 'new', 'england', '.', 'it', 'is', 'a', 'land', 'of', 'oil', ',', 'true', 'enough', ':']
['the', 'double-shuffle', '!', 'throw', 'yourselves', '!', 'legs', '!', 'legs', '!', '216', 'moby-dick', 'iceland', 'sailoe', '.', 'i', 'do', "n't", 'like', 'your', 'floor', ',', 'maty', ';', 'it', "'s", 'too', 'springy', 'to', 'my', 'taste', '.', 'i', "'m", 'used', 'to

[]

In [20]:
w2v.get_vector()

TypeError: 'KeyedVectors' object is not callable

In [197]:
layers.Softmax()(tf.ones((1,4)))

<tf.Tensor: shape=(1, 4), dtype=float32, numpy=array([[0.25, 0.25, 0.25, 0.25]], dtype=float32)>

In [239]:
class Attn(layers.Layer):
    def __init__(self,embed_dim):
        super(Attn, self).__init__()
        self.embed_dim = embed_dim
        self.query = layers.Dense(embed_dim)
        self.key = layers.Dense(embed_dim)
        self.value = layers.Dense(embed_dim)
        
        self.softmax = layers.Softmax()
    
    def call(self,inputs):
        values = self.value(inputs)
        queries = self.query(inputs)
        keys = self.key(inputs)
        
        #everything is batch_size,input_len,embed_dim
        
        
        similarity_scores = tf.matmul(queries,keys,transpose_b=True)/tf.sqrt(float(self.embed_dim))
        similarity_scores = self.softmax(similarity_scores)
        print(queries.shape, keys.shape)
        
        print(similarity_scores.shape,values.shape)
        
        out = tf.matmul(similarity_scores,values)
        return out
        
        
        

In [243]:
class rnn_attend(layers.Layer):
    def __init__(self,input_shape,vocab_size,attention_embed_dim=64):
        super(rnn_attend,self).__init__()
        self.bidirectional = layers.Bidirectional(layers.LSTM(10, return_sequences=True), input_shape=(5, 10))
        self.attn = Attn(attention_embed_dim)
        self.out = layers.Dense(vocab_size)
        
        self.bn1 = layers.BatchNormalization()
        
        
        
    def call(self, inputs):
        x = self.bidirectional(inputs)
        x = self.bn1(x)
        
        x = self.attn(x)
        x = self.out(x)
        